In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load CMp125 output

In [4]:
import numpy as np
import xarray as xr
from xgcm import Grid

In [48]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    ## Load CM4p125's state variable output
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/"
    umo = xr.open_mfdataset(f"{zdiag_path}ts/annual/10yr/*.umo.nc", chunks={'time':1})
    vmo = xr.open_mfdataset(f"{zdiag_path}ts/annual/10yr/*.vmo.nc", chunks={'time':1})
    trans = xr.merge([umo, vmo])

In [50]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_rho2.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})

    # correct state variable grids
    grid = Grid(trans, coords={'X': {'center': 'xh', 'outer': 'xq'},
                            'Y': {'center': 'yh', 'outer': 'yq'},
                            'Z': {'center': 'rho2_l', 'outer': 'rho2_i'} }, periodic=['X'])
    trans['geolon_c'] = xr.DataArray(data=ocean_grid['geolon_c'].data, dims=('yq', 'xq'))
    trans['geolat_c'] = xr.DataArray(data=ocean_grid['geolat_c'].data, dims=('yq', 'xq'))
    trans['geolon_u'] = xr.DataArray(data=ocean_grid['geolon_u'].data, dims=('yh', 'xq'))
    trans['geolat_u'] = xr.DataArray(data=ocean_grid['geolat_u'].data, dims=('yh', 'xq'))
    trans['geolon_v'] = xr.DataArray(data=ocean_grid['geolon_v'].data, dims=('yq', 'xh'))
    trans['geolat_v'] = xr.DataArray(data=ocean_grid['geolat_v'].data, dims=('yq', 'xh'))
    trans['geolon'] = xr.DataArray(data=ocean_grid['geolon'].data, dims=('yh', 'xh'))
    trans['geolat'] = xr.DataArray(data=ocean_grid['geolat'].data, dims=('yh', 'xh'))
    trans = trans.assign_coords({'geolon': trans['geolon'],
                                 'geolat': trans['geolat']})

In [51]:
rho2_interfaces = np.array([990.0, 1036.5, 1037.05, 1040.0])
rho2_levels = np.array([np.mean(rho2_interfaces[i:i+2]) for i in range(len(rho2_interfaces)-1)])

In [52]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    trans_wm = xr.Dataset({})
    trans_wm["umo"] = grid.transform(trans["umo"].fillna(0.), 'Z', rho2_interfaces, target_data=trans['rho2_i'], method="conservative")
    trans_wm["vmo"] = grid.transform(trans["vmo"].fillna(0.), 'Z', rho2_interfaces, target_data=trans['rho2_i'], method="conservative")
    trans_wm = trans_wm.rename({'rho2_i':'rho2_l'}).assign_coords({"rho2_l": rho2_levels})

In [62]:
for year, t in enumerate(trans_wm.time, start=1):
    trans_wm.sel(time=t).to_netcdf(f"/work/hfd/CM4-highres-WMT/data/abyssal_water_mass_transports_{str(year).zfill(4)}.nc", mode="w")